In [172]:
from pyspark.sql import SparkSession
from pyspark.sql import Row
from pyspark.sql.types import IntegerType

In [173]:
spark = SparkSession.builder.getOrCreate()
#specifying the data path
data_path = '/Users/hamidsakhi/git/BDM-P3/'
#specifying the file path
transaction_file_path = data_path + 'transaction.csv'
#loading the file into data frame
transaction_df = spark.read.format('csv').option('header','false').load(transaction_file_path)
#since the dataset doesn't have column names we should add headers here
transaction_df = transaction_df.withColumnRenamed('_c0','TransId')\
                               .withColumnRenamed('_c1','CustId')\
                               .withColumnRenamed('_c2','TransTotal')\
                               .withColumnRenamed('_c3','TransNumItem')\
                               .withColumnRenamed('_c4','TransDesc')
#checking if everything is fine so far
transaction_df.show()

+-------+------+----------+------------+---------------+
|TransId|CustId|TransTotal|TransNumItem|      TransDesc|
+-------+------+----------+------------+---------------+
|      1|  4902|   1258.52|           5|      YqAlzMxox|
|      2| 41719|   1335.05|           8|VjeZByKBUJOAjgU|
|      3| 22235|   1768.54|          14|          rPXFN|
|      4| 21358|    669.76|           5|ctquIBDNCPYAPZc|
|      5| 42597|    697.66|          12|         iHtMCY|
|      6| 39987|    863.77|           8| oSIjVjYzdWftFq|
|      7| 34951|    875.86|           5|         oItdbD|
|      8| 22142|   1738.62|           5|    rTmGTEHIAPH|
|      9| 15106|    541.77|           8|        UypGJFG|
|     10| 32555|    1371.1|           6|           Utuj|
|     11|  3292|    289.75|          14|           PMSp|
|     12| 16549|   1070.35|          14|        hwqbxhF|
|     13| 16800|    608.53|           3|            QWY|
|     14|  5849|    496.32|           7|           VZfy|
|     15| 10942|    666.45|    

In [174]:
#creating the view 
transaction_df.createOrReplaceTempView('purchases')

In [1]:
#T1: Filter out (drop) the Purchases from P with a total purchase amount above $600
T1 = spark.sql('SELECT * FROM purchases WHERE TransTotal>600')
transaction_df.createOrReplaceTempView("T1")
T1.show()

AnalysisException: Table or view not found: purchases; line 1 pos 14;
'Project [*]
+- 'Filter ('TransTotal > 600)
   +- 'UnresolvedRelation [purchases]


In [177]:
#T2: Over T1, group the Purchases by the Number of Items purchased, and for each group 
#calculate the median, min and max of total amount spent for purchases in that group.

T2 = spark.sql('SELECT TransNumItem, ROUND(AVG(TransTotal),2) AS Mean, ROUND(MIN(TransTotal),2)AS Min\
                ,ROUND(MAX(TransTotal),2) AS Max\
                FROM T1 \
               GROUP BY TransNumItem')
transaction_df.registerTempTable('T2')


In [109]:
#Report back your result T2 to the client side.
T2.show()

+------------+-------+-----+------+
|TransNumItem|   Mean|  Min|   Max|
+------------+-------+-----+------+
|           7|1004.88| 10.0|999.99|
|          15|1004.54| 10.0|999.99|
|          11|1004.05| 10.0|999.99|
|           3|1004.83| 10.0|999.99|
|           8| 1004.5|10.03|999.99|
|           5|1004.56|10.01|999.99|
|           6|1005.77|10.01|999.99|
|           9|1004.84| 10.0|999.99|
|           1| 1005.0| 10.0|999.97|
|          10| 1004.5|10.01|999.99|
|           4|1005.13| 10.0|999.99|
|          12|1004.87| 10.0|999.99|
|          13|1006.61|10.01|999.99|
|          14|1006.58| 10.0|999.99|
|           2|1004.65| 10.0|999.98|
+------------+-------+-----+------+



In [178]:
#T3T3: Over T1, group the Purchases from P by customer ID for young customers between 18 and 25 years of age, 
#and for each group report the customer ID, their age, and total number of items that this person has purchased 
#and total amount spent by the customer.

#for this we need to load the customer dataset
customer_file_path = data_path + 'customer.csv'
customer_df = spark.read.format('csv').option('header','false').load(customer_file_path)
#since the dataset doesn't have column names we should add headers here
customer_df = customer_df.withColumnRenamed('_c0','ID')\
                               .withColumnRenamed('_c1','Name')\
                               .withColumnRenamed('_c2','Age')\
                               .withColumnRenamed('_c3','CountryCode')\
                               .withColumnRenamed('_c4','Salary')
#creating the view 
customer_df.createOrReplaceTempView('customer')


In [185]:
#Now we can proceed joining the tables

customer = spark.sql("SELECT c.name from customer c")
customer.show()

+---------------+
|           name|
+---------------+
|  WivjZeovQMZrq|
|     EFwhtBzbOJ|
|   BURWswGeeprB|
|    auTTWZJzuGj|
|xmhQWsQlLbUzzqJ|
|  sQofCcIZOPPWm|
|     NmzngPDDSU|
|  otGJhOsmITyLM|
|     XxnYkBTqPn|
|NTTkOlsbNWLAVng|
|UUANxivXBuxiZXi|
|     AwjGAFKvED|
|    VYOpqwkNNeW|
|    dBjndbyWIen|
|    oUBGUlmPNiU|
| qYOXIzGZKtSZPg|
|TQmptUVxkikAuLo|
|voaLddVryIzGROI|
|   ULHKdwBUAMQf|
| OEgDoZXhKXwKmT|
+---------------+
only showing top 20 rows



In [187]:
t3 = spark.sql("""SELECT c.name, c.age, p.transtotal from customer c, purchases p
                WHERE c.id = p.custid
                  """)

In [188]:
t3.show()

Py4JJavaError: An error occurred while calling o3540.showString.
: org.apache.spark.SparkException: Exception thrown in awaitResult: 
	at org.apache.spark.util.ThreadUtils$.awaitResult(ThreadUtils.scala:226)
	at org.apache.spark.sql.execution.exchange.BroadcastExchangeExec.doExecuteBroadcast(BroadcastExchangeExec.scala:146)
	at org.apache.spark.sql.execution.InputAdapter.doExecuteBroadcast(WholeStageCodegenExec.scala:387)
	at org.apache.spark.sql.execution.SparkPlan$$anonfun$executeBroadcast$1.apply(SparkPlan.scala:144)
	at org.apache.spark.sql.execution.SparkPlan$$anonfun$executeBroadcast$1.apply(SparkPlan.scala:140)
	at org.apache.spark.sql.execution.SparkPlan$$anonfun$executeQuery$1.apply(SparkPlan.scala:155)
	at org.apache.spark.rdd.RDDOperationScope$.withScope(RDDOperationScope.scala:151)
	at org.apache.spark.sql.execution.SparkPlan.executeQuery(SparkPlan.scala:152)
	at org.apache.spark.sql.execution.SparkPlan.executeBroadcast(SparkPlan.scala:140)
	at org.apache.spark.sql.executio